In [ ]:
import pandas as pd
import numpy  as np

import random

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *

rt = RACETrack()

In [ ]:
svg  = '<svg x="0" y="0" width="512" height="512">'
svg += '<rect x="0" y="0" width="512" height="512" fill="#000000" />'
for i in range(0,8):
    if   i == 0:
        x0,y0,x1,y1 = 50,50,500,50
    elif i == 1:
        x0,y0,x1,y1 = 50,50,50,500
    elif i == 2:
        x0,y0,x1,y1 = 50,50,500,500
    else:
        x0,y0,x1,y1 = random.random()*512,random.random()*512,random.random()*512,random.random()*512
    svg += f'<line x1="{x0}" y1="{y0}" x2="{x1}" y2="{y1}" stroke="#808080" />'
    svg += rt.svgLabelOnLine(line=(x0,y0,x1,y1),color="#ffffff")
svg += '</svg>'
display(HTML(svg))

In [ ]:
#
# Test out the min and max precision based on user-supplied strings
# ... pay particular attention to roll-over dates
#
_tests = ['2021', 
          '2021-01', 
          '2022-06', 
          '2023-12',
          '2023-04-01',
          '2021-01-31',
          '2022-12-31',
          '2024-02-03 12',
          '2020-02-29 23',
          '2020-01-02 23:59',
          '2020-11-02 23:59',
          '2020-12-29 23:58',
          '2020-12-29 23:58:02',
          '2020-12-29 23:58:02.2',
          '2020-12-29 23:58:02.32',
          '2020-12-29 23:58:02.012',
          '2020-12-31 23:59:59'
]

for _test in _tests:
    _min,_max = rt.minTimeForStringPrecision(_test),rt.maxTimeForStringPrecision(_test)
    print(f'"{_test}"{" "*(32-len(_test))}"{_min}"{" "*(32-len(_min))}"{_max}"{" "*(16-len(_max))}')


In [ ]:
svg = '<svg x="0" y="0" width="1000" height="1000">'
svg += '<rect x="0" y="0" width="1000" height="1000" fill="#ff0000" />'

for x in range(0,800,250):
    for y in range(0,800,250):
        xchoice,ychoice = random.random(),random.random()

        if   xchoice < 0.33:
            xs = x + random.random()*75 - 5
            if   ychoice < 0.33:
                ys = y + random.random()*75
            elif ychoice < 0.66:
                ys = y+75+random.random()*100
            else:
                ys = y+75+100+random.random()*75
        elif xchoice < 0.66:
            xs = x+75+random.random()*100
            if   ychoice < 0.5:
                ys = y + random.random()*75 - 5
            else:
                ys = y+75+100+random.random()*75 + 5
        else:
            xs = x+75+100+random.random()*75 + 5
            if   ychoice < 0.33:
                ys = y + random.random()*75
            elif ychoice < 0.66:
                ys = y+75+random.random()*100
            else:
                ys = y+75+100+random.random()*75

        svg += rt.svgComicDialogue(_point=(xs, ys), _rect=(x+75,y+75,100,100))

svg += '</svg>'

display(HTML(svg))

In [ ]:
_background_lu = {}

_background_lu['poly_a'] = [
    (10,10),(40,10),(20,50),
    (40,100),(10,100)
]       

_background_lu['poly_b'] = [
    (200,200),(220,200),(220,180),(240,180),(240,200),(250,200),
    (250,10),(200,10)
]

_background_lu['poly_c'] = "M 40 40 L 100 40 L 100 100 Z M 250 250 L 280 250 L 280 280 L 250 280 Z M 50 300 L 100 300 L 100 250 Z"

_background_lu['pill'] = 'M 40 150 L 60 150 C 100 150 100 200 60 200 L 40 200 C 10 200 10 150 40 150 Z'

_sz = 300
_xcoords,_ycoords,_within = [],[],[]

_xcoords.append(0)
_ycoords.append(0)
_within.append(False)

_xcoords.append(_sz)
_ycoords.append(_sz)
_within.append(False)

for i in range(0,10000):
    _x,_y = random.random()*_sz,random.random()*_sz
    _xcoords.append(_x)
    _ycoords.append(_y)

    _within_something = False
    for _poly_key in _background_lu.keys():
        _polygon = _background_lu[_poly_key]
        if rt.pointWithinGeospatialBounds((_x,_y),_polygon):
            _within_something = True
    _within.append(_within_something)

_df = pd.DataFrame({'x':_xcoords,'y':_ycoords,'within':_within})

rt.co_mgr.str_to_color_lu[True]  = '#ff0000'
rt.co_mgr.str_to_color_lu[False] = '#808080'

rt.xy(_df, x_field='x', y_field='y', color_by='within', dot_size='small', opacity=0.4, bg_shape_lu=_background_lu, bg_shape_stroke='#ff0000')

In [ ]:
ts0 = ['2023-01-01 08:30', '2023-01-15 12:30', '2023-01-14 13:50', '2023-01-17 16:30', '2023-01-15']
ts1 = ['2023-01-01 09:30', '2023-01-17 16:30', '2023-01-16 01:15', '2023-01-18 20:10', '2023-01-15 23:59:59']
df = pd.DataFrame({'ts0':ts0,'ts1':ts1})
df = rt.columnsAreTimestamps(df,['ts0','ts1'])
df

In [ ]:
rt.removeAllAnnotations()
rt.addAnnotation(rt.eventAnnotation('test',timestamp_str='2023-01-16'))
rt.filterDataFrameToAnnotations(df, ts_field='ts0', ts_end_field='ts1')

In [ ]:
rt.removeAllAnnotations()
rt.addAnnotation(rt.eventAnnotation('test',timestamp_str='2023-01-01', timestamp_end_str='2023-01-31'))
rt.filterDataFrameToAnnotations(df, ts_field='ts0', ts_end_field='ts1')

In [ ]:
#
# Modeled after the following example:
# - http://hubbardworldstudies.weebly.com/annotated-timeline.html
# ... with information below taken from the corresponding wikipedia pages...
#
rt.removeAllAnnotations()
rt.addAnnotation(rt.eventAnnotation('George Müller',tags={'category':'famous_person'},
                                          timestamp_str='1805-09-27', timestamp_end_str='1898-03-10', 
                                          description_str='''George Müller was a Christian evangelist and the director of the Ashley Down orphanage in Bristol, England. He was one of the founders of the Plymouth Brethren movement. Later during the split, his group was called the Open Brethren.'''))
rt.addAnnotation(rt.eventAnnotation('Spanish-American War',tags={'category':'armed_conflict'},
                                          timestamp_str='1898-04-21', timestamp_end_str='1898-08-13',
                                          description_str='''The Spanish–American War began in the aftermath of the internal explosion of USS Maine in Havana Harbor in Cuba, leading to United States intervention in the Cuban War of Independence. The war led to the United States emerging predominant in the Caribbean region,[15] and resulted in U.S. acquisition of Spain's Pacific possessions. It led to United States involvement in the Philippine Revolution and later to the Philippine–American War.'''))
rt.addAnnotation(rt.eventAnnotation('Johann Baptist Strauss II',tags={'category':'famous_person'},
                                          timestamp_str='1825-10-25', timestamp_end_str='1899-06-03',
                                          description_str='''Johann Baptist Strauss II, also known as Johann Strauss Jr., the Younger or the Son (German: Johann Strauß Sohn), was an Austrian composer of light music, particularly dance music and operettas as well as a violinist. He composed over 500 waltzes, polkas, quadrilles, and other types of dance music, as well as several operettas and a ballet. In his lifetime, he was known as "The Waltz King", and was largely responsible for the popularity of the waltz in Vienna during the 19th century.'''))
rt.addAnnotation(rt.eventAnnotation('Theodore Roosevelt Jr.',tags={'category':'famous_person'},
                                          timestamp_str='1858-10-27', timestamp_end_str='1919-01-06',
                                          description_str='''Theodore Roosevelt Jr. (/ˈroʊzəvɛlt/ ROH-zə-velt;[b] October 27, 1858 – January 6, 1919), often referred to as Teddy or by his initials, T. R., was an American politician, statesman, soldier, conservationist, naturalist, historian, and writer who served as the 26th president of the United States from 1901 to 1909.'''))
rt.addAnnotation(rt.eventAnnotation('Philippine-American War',tags={'category':'armed_conflict'},
                                          timestamp_str='1899-02-04', timestamp_end_str='1902-07-02',
                                          description_str='''The Philippine–American War,[12] known alternatively as the Philippine Insurrection, Filipino–American War,[13] or Tagalog Insurgency,[14][15][16] was fought between the First Philippine Republic and the United States from February 4, 1899, until July 2, 1902.[17] Tensions arose after the United States annexed the Philippines under the Treaty of Paris at the conclusion of the Spanish–American War rather than acknowledging the Philippines' declaration of independence,[18][19]'''))
rt.addAnnotation(rt.eventAnnotation('Lyman Frank Baum',tags={'category':'famous_person'},
                                          timestamp_str='1856-05-15', timestamp_end_str='1919-05-06',
                                          description_str='''Lyman Frank Baum (/bɔːm/;[1] May 15, 1856 – May 6, 1919) was an American author best known for his children's books, particularly The Wonderful Wizard of Oz and its ten sequels. He wrote 14 novels in the Oz series, plus 41 other novels (not including four lost, unpublished novels), 83 short stories, over 200 poems, and at least 42 scripts. He made numerous attempts to bring his works to the stage and screen; the 1939 adaptation of the first Oz book became a landmark of 20th-century cinema.'''))
rt.addAnnotation(rt.eventAnnotation('Second Boer War',tags={'category':'armed_conflict'},
                                          timestamp_str='1899-10-11', timestamp_end_str='1902-05-31',
                                          description_str='''The Second Boer War (Afrikaans: Tweede Vryheidsoorlog, lit. 'Second Freedom War', 11 October 1899 – 31 May 1902), also known as the Boer War, the Anglo–Boer War, or the South African War, was a conflict which was fought between the British Empire and the two Boer Republics (the South African Republic and the Orange Free State) over the Empire's influence in Southern Africa from 1899 to 1902.'''))
rt.addAnnotation(rt.eventAnnotation('Lewis Carroll',tags={'category':'famous_person'},
                                          timestamp_str='1832-01-27', timestamp_end_str='1898-01-14',
                                          description_str='''Charles Lutwidge Dodgson (/ˈlʌtwɪdʒ ˈdɒdʒsən/ LUT-wij DOJ-sən; 27 January 1832 – 14 January 1898), better known by his pen name Lewis Carroll, was an English author, poet and mathematician. His most notable works are Alice's Adventures in Wonderland (1865) and its sequel Through the Looking-Glass (1871). He was noted for his facility with word play, logic, and fantasy. His poems Jabberwocky (1871) and The Hunting of the Snark (1876) are classified in the genre of literary nonsense.'''))
rt.addAnnotation(rt.eventAnnotation('Boxer Rebellion',tags={'category':'armed_conflict'},
                                          timestamp_str='1899', timestamp_end_str='1901',
                                          description_str='''The Boxer Rebellion, also known as the Boxer Uprising, the Boxer Insurrection, or the Yihetuan Movement, was an anti-foreign, anti-colonial, and anti-Christian uprising in China between 1899 and 1901, towards the end of the Qing dynasty, by the Society of Righteous and Harmonious Fists (Yìhéquán). The rebels were known as the "Boxers" in English because many of its members had practiced Chinese martial arts, which at the time were referred to as "Chinese boxing".'''))

timestamps,values = [],[]
for year in range(1800,1925):
    timestamps.append(f'{year:4}-01-01')
    values.append(random.randint(10,20))
df = pd.DataFrame({'timestamp':timestamps,'value':values})
df = rt.columnsAreTimestamps(df, ['timestamp'])

barchart_instance = rt.temporalBarChart(df, count_by='value',w=1024,h=128)

rt.annotateTimelineInstances(barchart_instance, max_line_w=300, annotation_color='#000000', max_lines=4)

In [ ]:
rt.annotateTimelineInstances(barchart_instance, max_line_w=240, max_lines=3, include_common_name=False)

In [ ]:
xy_instance = rt.xy(df, x_field='timestamp', y_field='value', w=1600, h=128)
rt.annotateTimelineInstances(xy_instance, max_line_w=300, max_lines=4, draw_text_border=True, annotation_color='common_name')

In [ ]:
rt.annotateTimelineInstances(xy_instance, max_line_w=200, max_lines=1, draw_text_border=True, annotation_color='tag:category')

In [ ]:
rt.timeline(timestamp='1800-01',timestamp_end='1805-01',w=1024)

In [ ]:
rt.timeline(df,ts_field='timestamp',w=512)

In [ ]:
rt.timeline(df,w=256)

In [ ]:
timeline_instance = rt.timeline(df, w=512, h=24)
rt.annotateTimelineInstances(timeline_instance, max_line_w=200, max_lines=1, draw_text_border=True, annotation_color='tag:category')

In [ ]:
timeline_instance = rt.timeline(df)
rt.annotateTimelineInstances(timeline_instance, max_line_w=200, max_lines=1, draw_text_border=True, annotation_color='tag:category')

In [ ]:
timeline_instance = rt.timeline(df)
rt.svgObject(rt.annotateTimelineInstancesSubOptimal(timeline_instance, max_line_w=200, max_lines=1, draw_text_border=True, annotation_color='tag:category'))